In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
from config import census_api_key, mysql_pw

ModuleNotFoundError: No module named 'config'

# Loading public health statistics for cities to dataframe

**Source:** Kaggle (https://www.kaggle.com/cdc/500-cities)

**Description:** This data was collected by Centers for Disease Control and Prevention, National Center for Chronic Disease Prevention and Health Promotion, Division of Population Health (2016)

**Data collection and preparation credit:** David Chen

In [ ]:
cdc_file = "data/500_Cities_CDC.csv"
cdc_data_df = pd.read_csv(cdc_file)
cdc_data_df.head()

In [ ]:
cdc_data_df.columns.tolist()

In [ ]:
column_list = ['ACCESS2_CrudePrev',
 'ACCESS2_Crude95CI',
 'ACCESS2_AdjPrev',
 'ACCESS2_Adj95CI',
 'ARTHRITIS_CrudePrev',
 'ARTHRITIS_Crude95CI',
 'ARTHRITIS_AdjPrev',
 'ARTHRITIS_Adj95CI',
 'BINGE_CrudePrev',
 'BINGE_Crude95CI',
 'BINGE_AdjPrev',
 'BINGE_Adj95CI',
 'BPHIGH_CrudePrev',
 'BPHIGH_Crude95CI',
 'BPHIGH_AdjPrev',
 'BPHIGH_Adj95CI',
 'BPMED_CrudePrev',
 'BPMED_Crude95CI',
 'BPMED_AdjPrev',
 'BPMED_Adj95CI',
 'CANCER_CrudePrev',
 'CANCER_Crude95CI',
 'CANCER_AdjPrev',
 'CANCER_Adj95CI',
 'CASTHMA_CrudePrev',
 'CASTHMA_Crude95CI',
 'CASTHMA_AdjPrev',
 'CASTHMA_Adj95CI',
 'CHD_CrudePrev',
 'CHD_Crude95CI',
 'CHD_AdjPrev',
 'CHD_Adj95CI',
 'CHECKUP_CrudePrev',
 'CHECKUP_Crude95CI',
 'CHECKUP_AdjPrev',
 'CHECKUP_Adj95CI',
 'CHOLSCREEN_CrudePrev',
 'CHOLSCREEN_Crude95CI',
 'CHOLSCREEN_AdjPrev',
 'CHOLSCREEN_Adj95CI',
 'COLON_SCREEN_CrudePrev',
 'COLON_SCREEN_Crude95CI',
 'COLON_SCREEN_AdjPrev',
 'COLON_SCREEN_Adj95CI',
 'COPD_CrudePrev',
 'COPD_Crude95CI',
 'COPD_AdjPrev',
 'COPD_Adj95CI',
 'COREM_CrudePrev',
 'COREM_Crude95CI',
 'COREM_AdjPrev',
 'COREM_Adj95CI',
 'COREW_CrudePrev',
 'COREW_Crude95CI',
 'COREW_AdjPrev',
 'COREW_Adj95CI',
 'CSMOKING_CrudePrev',
 'CSMOKING_Crude95CI',
 'CSMOKING_AdjPrev',
 'CSMOKING_Adj95CI',
 'DENTAL_CrudePrev',
 'DENTAL_Crude95CI',
 'DENTAL_AdjPrev',
 'DENTAL_Adj95CI',
 'DIABETES_CrudePrev',
 'DIABETES_Crude95CI',
 'DIABETES_AdjPrev',
 'DIABETES_Adj95CI',
 'HIGHCHOL_CrudePrev',
 'HIGHCHOL_Crude95CI',
 'HIGHCHOL_AdjPrev',
 'HIGHCHOL_Adj95CI',
 'KIDNEY_CrudePrev',
 'KIDNEY_Crude95CI',
 'KIDNEY_AdjPrev',
 'KIDNEY_Adj95CI',
 'LPA_CrudePrev',
 'LPA_Crude95CI',
 'LPA_AdjPrev',
 'LPA_Adj95CI',
 'MAMMOUSE_CrudePrev',
 'MAMMOUSE_Crude95CI',
 'MAMMOUSE_AdjPrev',
 'MAMMOUSE_Adj95CI',
 'MHLTH_CrudePrev',
 'MHLTH_Crude95CI',
 'MHLTH_AdjPrev',
 'MHLTH_Adj95CI',
 'OBESITY_CrudePrev',
 'OBESITY_Crude95CI',
 'OBESITY_AdjPrev',
 'OBESITY_Adj95CI',
 'PAPTEST_CrudePrev',
 'PAPTEST_Crude95CI',
 'PAPTEST_AdjPrev',
 'PAPTEST_Adj95CI',
 'PHLTH_CrudePrev',
 'PHLTH_Crude95CI',
 'PHLTH_AdjPrev',
 'PHLTH_Adj95CI',
 'SLEEP_CrudePrev',
 'SLEEP_Crude95CI',
 'SLEEP_AdjPrev',
 'SLEEP_Adj95CI',
 'STROKE_CrudePrev',
 'STROKE_Crude95CI',
 'STROKE_AdjPrev',
 'STROKE_Adj95CI',
 'TEETHLOST_CrudePrev',
 'TEETHLOST_Crude95CI',
 'TEETHLOST_AdjPrev',
 'TEETHLOST_Adj95CI']
print(column_list)

In [ ]:
len(column_list)

In [ ]:
for x in range (len(column_list)):
    if x % 4 == 2:
        print (column_list[x])
        

In [ ]:
new_cdc_data_df = cdc_data_df[["StateAbbr",
"PlaceName",
"PlaceFIPS",
"ACCESS2_AdjPrev",
"ARTHRITIS_AdjPrev",
"BINGE_AdjPrev",
"BPHIGH_AdjPrev",
"BPMED_AdjPrev",
"CANCER_AdjPrev",
"CASTHMA_AdjPrev",
"CHD_AdjPrev",
"CHECKUP_AdjPrev",
"CHOLSCREEN_AdjPrev",
"COLON_SCREEN_AdjPrev",
"COPD_AdjPrev",
"COREM_AdjPrev",
"COREW_AdjPrev",
"CSMOKING_AdjPrev",
"DENTAL_AdjPrev",
"DIABETES_AdjPrev",
"HIGHCHOL_AdjPrev",
"KIDNEY_AdjPrev",
"LPA_AdjPrev",
"MAMMOUSE_AdjPrev",
"MHLTH_AdjPrev",
"OBESITY_AdjPrev",
"PAPTEST_AdjPrev",
"PHLTH_AdjPrev",
"SLEEP_AdjPrev",
"STROKE_AdjPrev",
"TEETHLOST_AdjPrev"]].copy()
new_cdc_data_df.head()

In [ ]:
new_cdc_data_df_1 = new_cdc_data_df.rename(columns={"StateAbbr": "State",
"PlaceName": "City",
"PlaceFIPS": "PlaceFIPS",
"ACCESS2_AdjPrev": "ACCESS2",
"ARTHRITIS_AdjPrev" :"ARTHRITIS",
"BINGE_AdjPrev" :"BINGE",
"BPHIGH_AdjPrev" :"BPHIGH",
"BPMED_AdjPrev" :"BPMED",
"CANCER_AdjPrev" :"CANCER",
"CASTHMA_AdjPrev" :"CASTHMA",
"CHD_AdjPrev" :"CHD",
"CHECKUP_AdjPrev" :"CHECKUP",
"CHOLSCREEN_AdjPrev" :"CHOLSCREEN",
"COLON_SCREEN_AdjPrev" :"COLON_SCREEN",
"COPD_AdjPrev" :"COPD",
"COREM_AdjPrev" :"COREM",
"COREW_AdjPrev" :"COREW",
"CSMOKING_AdjPrev" :"CSMOKING",
"DENTAL_AdjPrev" :"DENTAL",
"DIABETES_AdjPrev" :"DIABETES",
"HIGHCHOL_AdjPrev" :"HIGHCHOL",
"KIDNEY_AdjPrev" :"KIDNEY",
"LPA_AdjPrev" :"LPA",
"MAMMOUSE_AdjPrev" :"MAMMOUSE",
"MHLTH_AdjPrev" :"MHLTH",
"OBESITY_AdjPrev" :"OBESITY",
"PAPTEST_AdjPrev" :"PAPTEST",
"PHLTH_AdjPrev" :"PHLTH",
"SLEEP_AdjPrev" :"SLEEP",
"STROKE_AdjPrev" :"STROKE",
"TEETHLOST_AdjPrev" :"TEETHLOST"    
})
new_cdc_data_df_1.head()

In [ ]:
place_fips = new_cdc_data_df_1["PlaceFIPS"].tolist()
print (place_fips)

In [ ]:
for y in place_fips:
    print (str(y)[-5:])

In [ ]:
new_cdc_data_df_1.to_csv("data/500_Cities_CDC_Disease_Columns.csv", encoding='utf-8',index= False)

In [ ]:
cdc_file_update_column = "data/500_Cities_CDC_Disease_Columns.csv"
cdc_data_df_update_column = pd.read_csv(cdc_file_update_column)
cdc_data_df_update_column.head()

# Loading Census Data

**Source:** Census API (https://api.census.gov)

**Description:** This data was called by county FIPS for all

**Census data collection and preparation:** Steve Bogdan

In [ ]:
url = "https://api.census.gov/data/2017/acs/acs5?get=B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B01001I_001E,B09020_001E,B15003_002E,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME&for=place:*&key=" + census_api_key

response = requests.get(url)
response_json = response.json()

census_df = pd.DataFrame(response_json)

census_df.columns = census_df.iloc[0]
census_df = census_df[1:]

census_df.head()

In [ ]:
column_names = {"B01003_001E":"total_pop", "B01001_020E":"male_65_66", "B01001_021E":"male_67_69",
                "B01001_022E":"male_70_74", "B01001_023E":"male_75_79", "B01001_024E":"male_80_84",
                "B01001_025E":"male_over_85", "B01001_044E":"female_65_66", "B01001_045E":"female_67_69",
                "B01001_046E":"female_70_74", "B01001_047E":"female_75_79", "B01001_048E":"female_80_84",
                "B01001_049E":"female_over_85", "B02001_002E":"white_pop", "B02001_003E":"black_pop",
                "B02001_004E":"native_amer_pop", "B02001_005E":"asian_pop", "B02001_006E":"pac_island_pop",
                "B02001_007E":"other_race_pop", "B01001I_001E":"hispanic_pop", "B15003_002E":"no_high_school",
                "B15003_017E":"high_school_grad", "B15003_022E":"bachelor_deg", "B15003_023E":"master_deg", 
                "B15003_025E":"doctorate_deg", "B17001_002E":"below_poverty", "B27001_002E":"male_w_health_ins",
                "B27001_030E":"female_w_health_ins", "place":"city_FIPS"}

census_df.rename(columns=column_names, 
                 inplace=True)


In [ ]:
for column in census_df.iloc[:, 0:29]:
    census_df[column] = pd.to_numeric(census_df[column])
    
census_df["with_health_ins"] = census_df["male_w_health_ins"] + census_df["female_w_health_ins"]
census_df.head()

In [ ]:
census_df["city_FIPS"] = census_df["state"] + census_df["city_FIPS"]
census_df.head()

In [ ]:
census_df["pop_over_65"] = census_df.iloc[:,1:13].sum(axis=1)
census_df["with_degree"] = census_df.iloc[:, 23:26].sum(axis=1)

In [ ]:
census_demographics_df=census_df[["total_pop", "with_health_ins", "pop_over_65", "white_pop", "black_pop", "native_amer_pop",
                                "asian_pop", "pac_island_pop", "other_race_pop", "hispanic_pop", "no_high_school",
                                "high_school_grad", "with_degree", "below_poverty", "city_FIPS"]]

census_demographics_df.head()

In [ ]:
for column in census_demographics_df.iloc[:, 1:14]:
    census_demographics_df[column] = census_demographics_df[column]/census_demographics_df.iloc[:,0]

census_demographics_df.head()

# DataBase Creation and Upload

We use MySQL database to store and aggregate the data

**Database creation:** Brian E Reyes, Olesya Bondarenko

In [ ]:
# Connect to MySQL
connection_string = f'root:{mysql_pw}@localhost/'
engine = create_engine(f'mysql://{connection_string}')
connection = engine.connect()

In [ ]:
# Create a new empty database
connection.execute('create schema city_health_demograhics;')

In [ ]:
# Create a connection to the new database
db_connection_string = f"root:{mysql_pw}@localhost/City_Health_Demographics"
db_engine = create_engine(f'mysql://{db_connection_string}')

In [ ]:
# Push the transformed CDC 500 US city data to the 'city_health_demographics' database. 
# It will be stored as a new table called 'city_health'
cdc_data_df_update_column.to_sql(name='City_Health', con=db_engine, if_exists='append', index=False)

In [ ]:
# Push the transformed Census data to the 'city_health_demographics' database. 
# It will be stored as a new table called 'city_demographics'
census_demographics_df.to_sql(name='City_Demographics', con=db_engine, if_exists='append', index=True)

In [ ]:
# Confirm tables
db_engine.table_names()

In [ ]:
# Perform an inner join on the city FIPS codes using MySQL query
# This results in creation of a new aggregated table, named 'city_health_demographics'
db_engine.execute('''create table city_health_demographics
                    select cd.*, ch.*
                    from city_demographics cd
                    inner join city_health ch
                    on cd.city_FIPS = ch.PlaceFIPS;''')